# Thura Tun

### The Sparks Foundation | GRIPJULY22 | 
### Computer vision & IoT Internship 2nd Task

### Task#4 - Detection of Face Mask

#### Notebook-file2: Apply mask-classification-model to a video file and save the output result

In [1]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from imutils.video import VideoStream

import numpy as np
import imutils
import time
import cv2
import os

2022-07-17 21:26:22.234087: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
def detect_and_predict_mask(frame, faceNet, maskNet):
    # grab dimension of frame
    (h, w) = frame.shape[:2]
    
    # construct a blob from the input frame
    blob = cv2.dnn.blobFromImage(frame, 1.0, (224, 224), (104.0, 117.0, 123.0))
    
    # forward pass inputBlob to FaceNet
    faceNet.setInput(blob)
    detections = faceNet.forward()
    
    # initialize the predictions requirements
    faces = []
    locs = []
    preds = []
    
    # loop over the detection
    for i in range(0, detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        
        # if confidence is greater than probability 0.5, get bbox locations
        if confidence > 0.5:
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype('int')
            
            face = frame[startY:endY, startX:endX]
            face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
            face = cv2.resize(face, (224, 224))
            face = img_to_array(face)
            face = preprocess_input(face)
            
            faces.append(face)
            locs.append((startX, startY, endX, endY))
    
    # if number of person is more than at least 1
    if len(faces) > 0:
        faces = np.array(faces, dtype='float32')
        preds = maskNet.predict(faces)
        
    return (locs, preds)

In [3]:
pwd

'/home/thura/Desktop/TSF-internship/detection-of-face-mask'

In [4]:
inputPath = '/home/thura/Desktop/TSF-internship/detection-of-face-mask/data/myMask1.mp4'
outPath = '/home/thura/Desktop/TSF-internship/detection-of-face-mask/output.avi'

video = cv2.VideoCapture(inputPath)
writer = None

In [5]:
# load face detector model from disk
protoPath = '/home/thura/Desktop/TSF-internship/detection-of-face-mask/face-detect-model/deploy.prototxt'
weightPath = '/home/thura/Desktop/TSF-internship/detection-of-face-mask/face-detect-model/res10_300x300_ssd_iter_140000.caffemodel'
faceNet = cv2.dnn.readNet(protoPath, weightPath)

maskModel = '/home/thura/Desktop/TSF-internship/detection-of-face-mask/mask-detector-model.model'
maskNet = load_model(maskModel)

print("[INFO] Starting Video....")

# Loop over the frame from video stream...
while True:
    ret, frame = video.read()
    
    if not ret:
        break
        
    else:
        frame = imutils.resize(frame, width=400)
        
        (locs, preds) = detect_and_predict_mask(frame, faceNet, maskNet)
        
        
        # loop over the detected face locations and their corresponding predictions
        for (bbox, preds) in zip(locs, preds):
            
            # unpack bounding-box and predictions
            (startX, startY, endX, endY) = bbox
            (mask, withoutMask) = preds
            
            # determine the class label and color we'll use to draw
            # the bounding box and text
            label = "Mask" if mask > withoutMask else "No Mask"
            color = (0, 255, 0) if label=="Mask" else (0, 0, 255)
            
            
            # include the probability in the label
            label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)
            
            
            # display the label and bounding box rectangle on the output
            # frame
            cv2.putText(frame, label, (startX, startY-10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
            cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)

    # actively show the frame        
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(10) & 0xFF

    # if 'q' key is pressed, break from the loop
    if key == ord('q'):
        break
    
    if writer is None:
        fourcc = cv2.VideoWriter_fourcc(*'XVID')
        writer = cv2.VideoWriter(outPath, fourcc, 25, 
                                 (frame.shape[1], frame.shape[0]), True)
        
    else:
        writer.write(frame)
        
writer.release()
cv2.destroyAllWindows()

2022-07-17 21:51:58.444328: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-07-17 21:51:58.585922: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-07-17 21:51:59.049639: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-17 21:51:59.068778: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:03:00.0 name: NVIDIA GeForce 920MX computeCapability: 5.0
coreClock: 0.993GHz coreCount: 2 deviceMemorySize: 1.96GiB deviceMemoryBandwidth: 13.41GiB/s
2022-07-17 21:51:59.076858: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-07-17 21:51:59.650498: I tensorflow/stream_executor/platform/default

[INFO] Starting Video....


2022-07-17 21:52:40.492895: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-07-17 21:52:40.646200: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2400000000 Hz
2022-07-17 21:52:42.560434: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2022-07-17 21:52:46.947220: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7
2022-07-17 21:52:57.275868: W tensorflow/stream_executor/gpu/asm_compiler.cc:63] Running ptxas --version returned 256
2022-07-17 21:52:57.317887: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] Internal: ptxas exited with non-zero error code 256, output: 
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
